In [ ]:
import pandas as pd
import numpy as np

from axisfuzzy.analysis.accessor import FuzzyAccessor
from axisfuzzy.analysis.app.model import Model
from axisfuzzy.analysis.component.basic import (
    ToolNormalization,
    ToolWeightNormalization,
    ToolFuzzification,
    ToolSimpleAggregation
)
from axisfuzzy.analysis.build_in import ContractCrispTable, ContractWeightVector
from axisfuzzy.fuzzifier import Fuzzifier # 假设 Fuzzifier 在这里

In [ ]:
# --- 1. 准备一个模糊化引擎 ---
# (假设 Fuzzifier 和 TriangularMF 已经可以正常工作)
my_fuzzifier = Fuzzifier(
    mf='gaussmf',
    mtype='qrohfn',  # 假设目标是 q-rung orthopair fuzzy number
    pi = 0.2,
    mf_params=[{'sigma': 0.15, 'c': 0.5}]
)

In [ ]:
# --- 2. 定义一个复杂模型 ---
class ComplexAnalysisModel(Model):
    """
    一个多输入、多输出的演示模型。
    - 输入: raw_data (DataFrame), raw_weights (vector)
    - 流程:
      1. 数据和权重分别进行归一化。
      2. 归一化后的数据进行模糊化。
      3. 对模糊化后的数据按行进行聚合（例如求均值）。
    - 输出: 聚合后的分数 (scores), 模糊化后的数据 (fuzzy_table)
    """

    def __init__(self):
        super().__init__()
        # 在 __init__ 中定义所有需要的“层”/组件
        self.data_normalizer = ToolNormalization(method='min_max', axis=0)
        self.weight_normalizer = ToolWeightNormalization()
        # 注意：ToolFuzzification 需要一个 Fuzzifier 实例
        # 这里我们假设一个简单的 Fuzzifier
        self.fuzzifier = ToolFuzzification(fuzzifier=my_fuzzifier)
        self.aggregator = ToolSimpleAggregation(operation='mean', axis=1)

    def get_config(self) -> dict:
        return {}

    def forward(self, data: ContractCrispTable, weights: ContractWeightVector):
        """
        用非常直观的方式定义数据流。
        这里的所有调用都将被自动翻译成 pipeline.add()。
        """
        # 分支 1: 处理数据
        norm_data = self.data_normalizer(data)
        fuzzy_table = self.fuzzifier(norm_data)

        # 分支 2: 处理权重 (在这个例子中，我们没有在下游使用它，但展示了分支能力)
        norm_weights = self.weight_normalizer(weights)

        # 聚合操作
        # scores = self.aggregator(fuzzy_table)

        # 返回一个包含多个输出的字典
        # return {'scores': scores, 'fuzzy_table': fuzzy_table}
        return {'fuzzy_table': fuzzy_table}

In [ ]:
# --- 3. 准备输入数据 ---
df = pd.DataFrame(np.random.rand(5, 3), columns=['Attr1', 'Attr2', 'Attr3'])
w = np.array([0.5, 0.3, 0.2])

In [ ]:
print("--- Initial Data ---")
print("DataFrame:\n", df)
print("Weights:\n", w)
print("-" * 20 + "\n")

In [ ]:
model = ComplexAnalysisModel()

In [ ]:
df.fuzzy.run(model, weights=w, return_intermediate=False)

In [ ]:
# --- 4. 实例化模型 ---
model = ComplexAnalysisModel()
model.build()
model.run(data=df, weights=w)

In [ ]:
model.summary()

In [ ]:
model.visualize()

In [ ]:
model.save('complex_model_demo.json')

In [ ]:
x = Model.load('complex_model_demo.json')

In [ ]:
x.build()

In [ ]:
x.summary()

In [ ]:
x.visualize()